In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np

tf.get_logger().setLevel('INFO')

2023-02-09 09:50:58.094910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-09 09:50:58.094943: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

x_train = train_df.drop(columns=["led_on"])
y_train = train_df["led_on"]

x_test = test_df.drop(columns=["led_on"])
y_test = test_df["led_on"]

x_train, y_train, x_test, y_test

(       pir  ldr_1  ldr_2   temp    hum   mic  timestamp
 0        1      0      1  26.24  28.24    39         17
 1        1      0      1  25.69  70.35   210         10
 2        1      1      1  26.46  39.70    11         19
 3        0      0      1  25.54  70.71   195         13
 4        1      1      1  24.48  47.13    11         20
 ...    ...    ...    ...    ...    ...   ...        ...
 20100    1      0      1  29.01  34.02   145         12
 20101    1      0      1  25.97  65.42  3827         10
 20102    1      1      1  26.87  55.61    81         21
 20103    0      0      1  29.68  29.93   219         15
 20104    0      1      1  27.20  53.83   122         20
 
 [20105 rows x 7 columns],
 0        0
 1        0
 2        1
 3        0
 4        1
         ..
 20100    0
 20101    0
 20102    1
 20103    0
 20104    1
 Name: led_on, Length: 20105, dtype: int64,
       pir  ldr_1  ldr_2   temp    hum   mic  timestamp
 0       1      0      1  26.47  57.11  3189         13

In [6]:
model = Sequential()
# add layers to the model
model.add(Dense(units=len(x_train.columns), activation='relu', input_shape=(len(x_train.columns),)))
model.add(Dense(units=len(x_train.columns), activation='relu'))
# output layer
model.add(Dense(units=1, activation='sigmoid'))
# compile the model
model.compile(loss='binary_crossentropy',
            optimizer='adam', metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 
                           tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives()])

In [8]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
629/629 [==============================] - 2s 2ms/step - loss: 0.2275 - accuracy: 0.9035 - precision: 0.6896 - recall: 0.8648 - true_negatives: 14966.0000 - false_positives: 1440.0000
Epoch 2/20
629/629 [==============================] - 2s 2ms/step - loss: 0.2267 - accuracy: 0.9024 - precision: 0.6872 - recall: 0.8619 - true_negatives: 14955.0000 - false_positives: 1451.0000
Epoch 3/20
629/629 [==============================] - 2s 2ms/step - loss: 0.2322 - accuracy: 0.8963 - precision: 0.6683 - recall: 0.8662 - true_negatives: 14816.0000 - false_positives: 1590.0000
Epoch 4/20
629/629 [==============================] - 1s 2ms/step - loss: 0.2244 - accuracy: 0.9047 - precision: 0.6914 - recall: 0.8708 - true_negatives: 14968.0000 - false_positives: 1438.0000
Epoch 5/20
629/629 [==============================] - 1s 2ms/step - loss: 0.2294 - accuracy: 0.9008 - precision: 0.6806 - recall: 0.8681 - true_negatives: 14899.0000 - false_positives: 1507.0000
Epoch 6/20
629/629 [=====

In [9]:
model.evaluate(x_test, y_test)

158/158 [==============================] - 1s 2ms/step - loss: 0.2183 - accuracy: 0.9039 - precision: 0.7154 - recall: 0.8328 - true_negatives: 3737.0000 - false_positives: 321.0000


[0.21825173497200012,
 0.9039188623428345,
 0.7154255509376526,
 0.8328173160552979,
 3737.0,
 321.0]

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("mlp.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpdila2w41/assets


INFO:tensorflow:Assets written to: /tmp/tmpdila2w41/assets
2023-02-09 10:02:45.639408: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-09 10:02:45.639461: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-09 10:02:45.639647: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpdila2w41
2023-02-09 10:02:45.641015: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-02-09 10:02:45.641040: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpdila2w41
2023-02-09 10:02:45.648888: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-02-09 10:02:45.714054: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpdila2w41
2023-02-09 10:02:45.734080: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

2516